In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import math

In [5]:
class Rectangle:
    
    def rotate(self, angle, vertex=0):
        '''
        Rotate around fulcrum vertex for *angle* degrees
        '''
        # update sides' angles
        self.angles += math.radians(angle)
        
        if vertex < 0 or vertex > 3: 
            raise ValueError('vertex must be between 0 and 3')
        if   vertex%2 == 0: i, j = self.width, self.height
        elif vertex%2 == 1: i, j = self.height, self.width
        
        # indices rearranged beginning from fulcrum vertex
        ind = [*[i for i in range(vertex, 4)], *[i for i in range(0, vertex)]]
        points = self.points[ind]
        angles = self.angles[ind]
        
        # rotation movement performed from first 3 to last 3 vertices
        rotation = np.hstack([np.cos(angles[:-1, np.newaxis]), 
                              np.sin(angles[:-1, np.newaxis])])*np.array([i,j,i])[:, np.newaxis]
        
        for q in range(0,3): points[q+1] = points[q] + rotation[q]   
        self.points[ind] = points
    
    def __init__(self, vertex, width, height, angle=0):
        '''
                          D____C
                          |    |
        Rectangle of type A____B
        '''
        self.width  = width
        self.height = height
        
        # matrix of unrotated points
        self.points = np.array([vertex[0], vertex[1]]*4
                              ).astype('float64' # force float division
                              ).reshape((4, 2))
        self.points[1:3, 0] += width
        self.points[2:4, 1] += height
        
        # angles of every side with positive x axis (AB, BC, CD, DA)
        self.angles = np.array([math.radians(x) for x in [angle, angle+90, angle+180, angle-90]])
        self.rotate(angle, 0)
        
    def plot(self, xlim=None, ylim=None, figsize=(5,5)):
        fig, ax = plt.subplots(figsize=figsize)
        if xlim != None: ax.set_xlim(*xlim)
        if ylim != None: ax.set_ylim(*ylim)
            
        ax.plot(self.points[[0,1,2,3,0],0], self.points[[0,1,2,3,0],1])
        return fig, ax
    
    def min_max(self, x):
        '''
        Find min and max y of Rectangle for given x.
        Compute min_max fo points that lie on y but not x limits of Rectangle (with consequences for
        includes and collides_with methods)
        '''
        xs = self.points[:,0]
        ys = self.points[:,1]
        
        if not np.min(xs) < x < np.max(xs): return None
        
        # rows indices where x is included between two clockwise successive vertices
        ind = np.append(np.arange(4)[(xs < x) & (xs[[1,2,3,0]] > x)],
                        np.arange(4)[(xs > x) & (xs[[1,2,3,0]] < x)]) # same but in counter clockwise sense
        
        m1 = (ys[ind[0]+1] - ys[ind[0]])/(xs[ind[0]+1] - xs[ind[0]]) # slope
        q1 = ys[ind[0]] - m1*xs[ind[0]]                              # intercept
        res = [x*m1+q1]*2
        
        if len(ind) == 2:
            m2 = (ys[ind[1]] - ys[ind[1]+1])/(xs[ind[1]] - xs[ind[1]+1]) # slope
            q2 = ys[ind[1]+1] - m2*xs[ind[1]+1]                          # intercept
            res[1] = x*m2+q2
        
        return res
    
    def includes(self, point):
        '''
        For point[x,y] return True if point is included in Rectangle
        '''
        x, y = point[0], point[1]
        
        try: 
            minim, maxim = self.min_max(x)
            if minim <= y <= maxim: return True
        except TypeError: return False
        return False
    
    def collides_with(self, rect):
        '''
        Return True if self collides with rect
        '''
        for i in range(4):
            if self.includes(rect.points[i]) or rect.includes(self.points[i]): return True
        return False